In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("data/raw/uk-war.csv", names=["date","time","location","lat","lon","kind","description","link"])
df.dropna(axis=0,how='any',subset=['lat','lon'],inplace=True)
# Variable with the Longitude and Latitude
X=df.loc[:,['kind','lat','lon']]
X = X[X["kind"].str.contains("Ua") == False]
X["party"]=0
Y=df.loc[:,['kind','lat','lon']]
Y = Y[Y["kind"].str.contains("Ru") == False]
Y = Y[Y["kind"].str.contains("RU") == False]
Y["party"] = 1

In [ ]:
frames = [X, Y]
Z = pd.concat(frames)
Z


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# Fit only on X_train
scaler.fit(Z[Z.columns[1:3]])

# Scale both X_train and X_test
Z_scaled = scaler.transform(Z[Z.columns[1:3]])


scaled_df = pd.DataFrame(Z_scaled, columns=["lat","lon"])

scaled_df


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
regressor = KNeighborsRegressor(n_neighbors = 3, metric = 'minkowski', p = 2)
regressor.fit(scaled_df, Z.party)

In [ ]:
y_pred = regressor.predict(scaled_df)
y_pred

In [ ]:
from matplotlib.colors import ListedColormap
import numpy as np
X_set, y_set = Z_scaled, Z.party
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, regressor.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('KNN Classification')
plt.xlabel('DMV_Test_1')
plt.ylabel('DMV_Test_2')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (15,5))
plt.scatter(Z.lon, Z.lat, c=y_pred, marker= 'o', s=100,edgecolors='red')

In [ ]:
Z.lat.min()

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)
indices




